# <center> Run custom Spark on Palmetto instead of Cypress </center>

# Required Modules 
* hdp/0.1
* anaconda3/5.1.0

In [1]:
!module list 

Currently Loaded Modulefiles:
  1) gcc/7.1.0         3) use.own           5) hdp/0.1
  2) vim/8.0.1763      4) anaconda3/5.1.0


In [2]:
import sys
print(sys.version_info)
assert(sys.version.split('|')[0].replace(' ','') == '3.6.4'), "Wrong python version for pyspark"

sys.version_info(major=3, minor=6, micro=4, releaselevel='final', serial=0)


In [3]:
!cypress-kinit
!klist

Ticket cache: FILE:/home/esadrfa/.krb5cc
Default principal: esadrfa@PALMETTO.CLEMSON.EDU

Valid starting       Expires              Service principal
07/31/2018 12:46:46  08/07/2018 12:46:46  krbtgt/PALMETTO.CLEMSON.EDU@PALMETTO.CLEMSON.EDU


## 1. Getting Started

Spark stores data in memory. This memory space is represented by variable **sc** (SparkContext). 

In [4]:
import sys
import os

sys.path.insert(0, '/usr/hdp/current/spark2-client/python')
sys.path.insert(0, '/usr/hdp/current/spark2-client/python/lib/py4j-0.10.6-src.zip')

os.environ['SPARK_HOME'] = '/usr/hdp/current/spark2-client/'
os.environ['SPARK_CONF_DIR'] = '/etc/hadoop/synced_conf/spark2/'
os.environ['PYSPARK_PYTHON'] = '/software/anaconda3/5.1.0/bin/python'

# https://spoddutur.github.io/spark-notes/distribution_of_executors_cores_and_memory_for_spark_application.html
    
import pyspark
conf = pyspark.SparkConf()
conf.setMaster("yarn")
conf.set("spark.driver.memory","1700g")  #request a little less than what we asked from palmetto (6g)
conf.set("spark.driver.maxResultSize", "120g")
conf.set("spark.executor.memory","80g")   # 60g per instance of executer
conf.set("spark.executor.instances", "120")
conf.set("spark.executor.cores","5")       # more than 5 bug down the HDFS
conf.set("spark.rpc.message.maxSize", "1024")
conf.set("spark.driver.maxResultSize", "1000g")
# https://stackoverflow.com/questions/35090222/spark-lda-woes-prediction-and-oom-questions
# https://spark.apache.org/docs/latest/tuning.html
conf.set("spark.rdd.compress","true")
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")

# I am not sure about the below ones 
# https://spark.apache.org/docs/latest/configuration.html
# conf.set("spark.memory.offheap.enabled", "true")
# conf.set("spark.memory.offHeap.size, "20"
sc = pyspark.SparkContext(conf=conf)

In [2]:
sc

''

In [1]:
sc.stop()

NameError: name 'sc' is not defined

In [5]:
print(sc.version)
sqlContext = pyspark.SQLContext(sc)

2.3.0.2.6.5.0-292


In [6]:
from multiprocessing.dummy import Pool as ThreadPool 
import pickle
import numpy as np
from pyspark.ml.linalg import SparseVector

## the connection to HPC failed over the weekend, I restart the loading be initializing the required variables

In [7]:
num_range = 4612
data_path = 'HypoClas/'
scaled_transposed_df_parts_folder = '/SO_CS_EU_s0_e99999_scaled_transposed_part'
report_interval = int(num_range / 100)
report_interval

46

In [8]:
lst_dfs = [None] * num_range
selected_indices_dfs = [i for i in range(num_range)]

def loadDataFrame(idx):
    lst_dfs[idx] = sqlContext.read.format('parquet')\
               .load(data_path + scaled_transposed_df_parts_folder + str(idx))

In [9]:
%%time
pool = ThreadPool() 
pool.map(loadDataFrame, selected_indices_dfs)

pool.close()
pool.join()

CPU times: user 2.45 s, sys: 526 ms, total: 2.97 s
Wall time: 1min 54s


In [10]:
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

# https://datascience.stackexchange.com/a/11361/8723
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [11]:
%%time
raw_df = unionAll(*lst_dfs)

CPU times: user 968 ms, sys: 252 ms, total: 1.22 s
Wall time: 5min 59s


In [12]:
print(raw_df.columns)
print(type(raw_df))

['id', 'features']
<class 'pyspark.sql.dataframe.DataFrame'>


In [13]:
%%time
# https://stackoverflow.com/a/37286339/2674061
raw_rdd = raw_df.select('features').rdd.map(list)

CPU times: user 16.3 ms, sys: 5.36 ms, total: 21.7 ms
Wall time: 1min 47s


In [14]:
%%time
raw_rdd.cache()  # this just set a flag for next time, I run it 
# raw_rdd.persist()  # 
raw_rdd.count()  # to cache it 

CPU times: user 661 µs, sys: 3.18 ms, total: 3.84 ms
Wall time: 21.3 ms


PythonRDD[18451] at RDD at PythonRDD.scala:48

In [15]:
%%time
from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix, RowMatrix
from pyspark.ml.linalg import SparseVector

mat = RowMatrix(raw_rdd)

CPU times: user 23.2 ms, sys: 5.28 ms, total: 28.5 ms
Wall time: 111 ms


In [16]:
%%time
# Compute the top k singular values and corresponding singular vectors.
svd = mat.computeSVD(1000, computeU=True)

KeyboardInterrupt: 

In [17]:
mat